# Model evaluation

Now that we have trained our model, we need to evaluate its performance. In order to do this we will use the test set that we separated in the beginning and also some images from the internet

> **Note**: This notebook is meant to be used with Kaggle notebooks (and also Colab), the link is [here](). From Kaggle the notebook can be opened in Colab.

### Install detectron2 and import the required pakages

In [ ]:
# Install detectron2
!python -m pip install pyyaml==5.1
import sys, os, shutil, distutils.core
if os.path.isdir('./detectron2'):
    shutil.rmtree("./detectron2")
# Note: This is a faster way to install detectron2 in Colab, but it does not include all functionalities (e.g. compiled operators).
# See https://detectron2.readthedocs.io/tutorials/install.html for full installation instructions
!git clone 'https://github.com/facebookresearch/detectron2'
dist = distutils.core.run_setup("./detectron2/setup.py")
!python -m pip install {' '.join([f"'{x}'" for x in dist.install_requires])}
sys.path.insert(0, os.path.abspath('./detectron2'))

# Get pytorch and CUDA vertions
import detectron2, torch
TORCH_VERSION = ".".join(torch.__version__.split(".")[:2])
CUDA_VERSION = torch.__version__.split("+")[-1]

# Setup the logger
from detectron2.utils.logger import setup_logger
setup_logger()

# import some common libraries
import numpy as np
import os, random
import matplotlib.pyplot as plt

# import some common detectron2 utilities
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
from detectron2.data import build_detection_test_loader

### Register the datasets

The validation dataset, with the images created with UnrealCV, and the test dataset, with the images from the internet, are registered.

In [2]:
from detectron2.data.datasets import register_coco_instances
register_coco_instances("person_park_val", {}, "/kaggle/input/personparkval/dataset_validation.json", "/kaggle/input/personparkval/lit/lit")
register_coco_instances("person_park_test", {}, "/kaggle/input/personparktest/dataset_test.json", "/kaggle/input/personparktest/imgs")

### Get the parameters and weights from the trained model

In [ ]:

cfg = get_cfg()
# TODO: Get this from the custom model
from detectron2 import model_zoo
cfg.merge_from_file(model_zoo.get_config_file("COCO-Detection/faster_rcnn_R_101_FPN_3x.yaml"))
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 1

cfg.MODEL.WEIGHTS = "/kaggle/input/personparkmodel/pytorch/peoplepark/1/model_final.pth"


## Qualitative evaluation

Let's visualize the predictions made by the model. First we need to set a threshold for the confidence of the predictions.

In [ ]:
# set a custom testing threshold
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.8  
# Create the predictor
predictor = DefaultPredictor(cfg)

Get a random image from the validation dataset and visualize the predictions made by the model.

In [ ]:
# take a random image
validation_imgs = os.listdir("/kaggle/input/personparkval/lit/lit/");
random_img = random.choice(validation_imgs)
im = plt.imread("/kaggle/input/personparkval/lit/lit/%s" % random_img)

# make a prediction
metadata = MetadataCatalog.get("person_park_val")
outputs = predictor(im)
v = Visualizer(im,
               metadata=metadata, 
               scale=0.5,
)

# draw the prediction
out = v.draw_instance_predictions(outputs["instances"].to("cpu"))
plt.imshow(out.get_image()[:, :, ::-1])

This is the result:

![predictions]()

We can to the same thing with random images taken from the internet

In [ ]:
# take a random image
validation_imgs = os.listdir("/kaggle/input/personparktest/imgs");
random_img = random.choice(validation_imgs)
im = cv2.imread("/kaggle/input/personparktest/imgs/%s" % random_img)

# make a random image
metadata = MetadataCatalog.get("person_park_test")
outputs = predictor(im)
v = Visualizer(im,
               metadata=metadata, 
               scale=0.5,
)

# draw the prediction
out = v.draw_instance_predictions(outputs["instances"].to("cpu"))
plt.imshow(out.get_image()[:, :, ::-1])

Some examples:
|   |   |
|---|---|
![]()   |  ![]()

We can compare this prediction with the model we started with

Some examples:
| Custom model | Faster-RCNN tained on imageNet  |
|---|---|
![]()   |  ![]()

# Quantitative evaluation

We can use detectron2 to evaluate the model with the COCO evaluation metric

First evaluate the model with the validation dataset

In [ ]:
evaluator = COCOEvaluator("person_park_val")
val_loader = build_detection_test_loader(cfg, "person_park_val")
print(inference_on_dataset(predictor.model, val_loader, evaluator))

Evaluate the custom model and the model from detectron2 model zoo with the test dataset 

In [ ]:
setup_logger()
evaluator = COCOEvaluator("person_park_test")
val_loader = build_detection_test_loader(cfg, "person_park_test")
print(inference_on_dataset(predictor.model, val_loader, evaluator))